## Convert Image Data to COCO Json format

In [1]:
import json
import numpy as np
from PIL import Image
from collections import defaultdict
import cv2
from skimage import measure,io
import os
from skimage.color import rgb2gray
from pycocotools import mask as mask_utils


def generate_annotations(image_folder, mask_folder,categories = None):
    images = []
    annotations = []
    categories = categories

    image_id = 1
    annotation_id = 1

    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)
            mask_path = os.path.join(mask_folder, filename.replace(".jpg", ".png"))  # Assuming mask images are in PNG format

            image = cv2.imread(image_path)
            height, width, _ = image.shape

            images.append({
                "file_name": filename,
                "height": height,
                "width": width,
                "id": image_id
            })

            labels = cv2.imread(mask_path)
            gray_mask = cv2.cvtColor(labels, cv2.COLOR_BGR2GRAY)
            unique_colors = np.unique(gray_mask)    
            
            for color in unique_colors:
                if color == 255:
                    continue  # Ignore background color
                category_id = int(color)  # Assuming pixel value directly corresponds to category ID
                binary_mask = np.uint8(gray_mask == category_id) * 255
                               # Find contours on the mask
                contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                for contour in contours:
                    if len(contour) < 3:
                        continue  # Skip contours with fewer than 3 points
                    area = float(cv2.contourArea(contour))
                    x, y, w, h = cv2.boundingRect(contour)
                    bbox = [float(x), float(y), float(w), float(h)]

                    annotation = {
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": category_id,
                        "segmentation": [contour.astype(int).flatten().tolist()],
                        "bbox": bbox,
                        "area": area,
                        "iscrowd": 0
                    }
                    annotations.append(annotation)
                    annotation_id += 1
    
            image_id += 1

    return {"images": images, "annotations": annotations, "categories": categories}

def save_annotations(annotations, output_path):
    with open(output_path, "w") as outfile:
        json.dump(annotations, outfile)



In [2]:
categories = [
    {"id": 1, "name": "sky", "supercategory": "object"},
    {"id": 2, "name": "ridge", "supercategory": "object"},
    {"id": 3, "name": "soil", "supercategory": "object"},
    {"id": 4, "name": "sand", "supercategory": "object"},
    {"id": 5, "name": "bedrock", "supercategory": "object"},
    {"id": 6, "name": "rock", "supercategory": "object"},
    {"id": 7, "name": "rover", "supercategory": "object"},
    {"id": 8, "name": "trace", "supercategory": "object"},
    {"id": 9, "name": "hole", "supercategory": "object"},
]

image_folder = './combine_dataset/final_dataset/train/images'
mask_folder = './combine_dataset/final_dataset/train/masks'

# Create COCO JSON for instance segmentation
output_train_file  = './instance_coco_annotations_train.json'
annotations = generate_annotations(image_folder, mask_folder,categories=categories)
save_annotations(annotations, output_train_file)

In [3]:
categories = [
    {"id": 1, "name": "sky", "supercategory": "object"},
    {"id": 2, "name": "ridge", "supercategory": "object"},
    {"id": 3, "name": "soil", "supercategory": "object"},
    {"id": 4, "name": "sand", "supercategory": "object"},
    {"id": 5, "name": "bedrock", "supercategory": "object"},
    {"id": 6, "name": "rock", "supercategory": "object"},
    {"id": 7, "name": "rover", "supercategory": "object"},
    {"id": 8, "name": "trace", "supercategory": "object"},
    {"id": 9, "name": "hole", "supercategory": "object"},
]

image_folder = './combine_dataset/final_dataset/val/images'
mask_folder = './combine_dataset/final_dataset/val/masks'

# Create COCO JSON for instance segmentation
output_val_file  = './instance_coco_annotations_val.json'
annotations = generate_annotations(image_folder, mask_folder,categories=categories)
save_annotations(annotations, output_val_file)               

In [4]:
categories = [
    {"id": 1, "name": "sky", "supercategory": "object"},
    {"id": 2, "name": "ridge", "supercategory": "object"},
    {"id": 3, "name": "soil", "supercategory": "object"},
    {"id": 4, "name": "sand", "supercategory": "object"},
    {"id": 5, "name": "bedrock", "supercategory": "object"},
    {"id": 6, "name": "rock", "supercategory": "object"},
    {"id": 7, "name": "rover", "supercategory": "object"},
    {"id": 8, "name": "trace", "supercategory": "object"},
    {"id": 9, "name": "hole", "supercategory": "object"},
]

image_folder = './combine_dataset/final_dataset/test/images'
mask_folder = './combine_dataset/final_dataset/test/masks'

# Create COCO JSON for instance segmentation
output_test_file  = './instance_coco_annotations_test.json'
annotations = generate_annotations(image_folder, mask_folder,categories=categories)
save_annotations(annotations, output_test_file)